In [19]:
import pickle
import sys
import nltk
from pomegranate import *
from collections import defaultdict
import numpy as np 
from sklearn.model_selection import KFold

In [20]:
with open('pair.pkl','rb') as f:
    pair = pickle.load(f)[0:50000]

In [ ]:
pair.append([('UNK','N')])
data = np.array(pair)

In [ ]:
Kf = KFold(n_splits=5)
accr = []
for train, test in Kf.split(np.array(pair)):
    trn = data[train]
    tst = data[test]
    ####TRAIN####
    w_trn = [] #所有出现的词
    p_trn = [] #所有词性
    for i in trn:
        for j in i:
            w_trn.append(j[0])
            p_trn.append(j[1])
    
    w_p_trn = defaultdict(list)
    for i in range(len(w_trn)):
        w_p_trn[w_trn[i]].append(p_trn[i])

    #每个词性对应list of words
    p_w_trn = defaultdict(list)
    for i in range(len(w_trn)):
        p_w_trn[p_trn[i]].append(w_trn[i])


    w_dic2_trn = {}
    for i in w_p_trn:
        p_dic2_trn = {}
        for j in w_p_trn[i]:
            p_dic2_trn[j]=w_p_trn[i].count(j)
        w_dic2_trn[i] = p_dic2_trn
    #print(w_dic2)


    p_dic1_trn = {}
    for i in p_w_trn:
        w_dic1_trn = {}
        for j in p_w_trn[i]:
            w_dic1_trn[j]=p_w_trn[i].count(j)
        p_dic1_trn[i] = w_dic1_trn
        
    #emission matrix 
    p_freq_trn = {}
    for i in p_dic1_trn:
        p_count_trn = 0
        word_trn = {}
        for j in p_dic1_trn[i]:
            p_count_trn += p_dic1_trn[i][j]
        for j in p_dic1_trn[i]:
            word_trn[j]=p_dic1_trn[i][j]/p_count_trn
        p_freq_trn[i] = State(DiscreteDistribution(word_trn),name = i)

    model_trn = HiddenMarkovModel('pos_tag_trn')

    #共有词性多少个
    count_trn = 0
    for i in p_dic1_trn:
        for j in p_dic1_trn[i]:
            count_trn += p_dic1_trn[i][j]

    states_trn = [] #pos tag
    for i in p_freq_trn:
        states_trn.append(p_freq_trn[i])

    model_trn.add_states(states_trn)

    for i in p_freq_trn:
        model_trn.add_transition(model_trn.start, p_freq_trn[i], p_trn.count(i)/count_trn)

    #transition matrix
    p_sentence_trn =[]
    w_sentence_trn = []
    for i in trn:
        p_string_trn = []
        w_string_trn = []
        for j in i:
            p_string_trn.append(j[1])
            w_string_trn.append(j[0])
        p_sentence_trn.append(p_string_trn)
        w_sentence_trn.append(w_string_trn)

    trans_prob_trn = {}
    for s in p_sentence_trn:
        numerator_trn =0
        for i in range(len(s)-1):
            first_trn = s[i]
            second_trn = s[i+1]
            denom_trn = p_trn.count(first_trn)
        if any([first_trn,second_trn] == s[i:i+2] for i in range(len(s) - 1)):
            numerator_trn += 1
        trans_prob_trn[(first_trn,second_trn)] = numerator_trn/denom_trn


    for i in p_freq_trn:
        prob_trn=0
        for j in p_freq_trn:
            if (i,j) not in trans_prob_trn.keys():
                trans_prob_trn[(i,j)] = 0
            model_trn.add_transition(p_freq_trn[i],p_freq_trn[j],trans_prob_trn[(i,j)])

            prob_trn+=trans_prob_trn[(i,j)]
        if prob_trn<1:
            model_trn.add_transition(i,model_trn.end,1-prob_trn)
    model_trn.bake()
    
    ####TEST####
    #POS sentence list in test data
    p_sentence_tst=[]
    #word sentence list in test data
    w_sentence_tst=[]
    for i in tst:
        p_string_tst = []
        w_string_tst = []
        for j in i:
            p_string_tst.append(j[1])   
            w_string_tst.append(j[0])
        p_sentence_tst.append(p_string_tst)
        w_sentence_tst.append(w_string_tst)
    
    correct = 0
    tot = 0
    for i in range(len(w_sentence_tst)):
        tot+=len(w_sentence_tst[i]) #一句话的字符数量
        for j in range(len(w_sentence_tst[i])):     
            if w_sentence_tst[i][j] not in w_trn:
                w_sentence_tst[i][j] = 'UNK'
        pred = [state.name for q,state in model_trn.viterbi(w_sentence_tst[i])[1]]
        pred.remove('pos_tag-start')
        for pos in range(len(pred)):
            if pred[pos] == p_sentence_tst[i][pos]:
                correct += 1
         
    accr.append(correct/tot)

print(mean(accr))


In [31]:
X = [1,2,3,4,5,6,7,8,9,9]
X.remove(9)
X  

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
namae = [state.name for i,state in model.viterbi(['这个','词','的','意思','是'])[1]]
namae.remove('pos_tag-start')
print(namae)

In [ ]:
#################################################################################################################

In [ ]:
def HMM(data):
    w_data = [] #所有出现的词
    p_data = [] #所有词性
    for i in data:
        for j in i:
            w_data.append(j[0])
            p_data.append(j[1])
    
    w_p_data = defaultdict(list)
    for i in range(len(w_data)):
        w_p_data[w_data[i]].append(p_data[i])

    #每个词性对应list of words
    p_w_data = defaultdict(list)
    for i in range(len(w_data)):
        p_w_data[p_data[i]].append(w_data[i])


    w_dic2_data = {}
    for i in w_p_data:
        p_dic2_data = {}
        for j in w_p_data[i]:
            p_dic2_data[j]=w_p_data[i].count(j)
        w_dic2_data[i] = p_dic2_data
    #print(w_dic2)


    p_dic1_data = {}
    for i in p_w_data:
        w_dic1_data = {}
        for j in p_w_data[i]:
            w_dic1_data[j]=p_w_data[i].count(j)
        p_dic1_data[i] = w_dic1_data
        
    #emission matrix 
    p_freq_data = {}
    for i in p_dic1_data:
        p_count_data = 0
        word_data = {}
        for j in p_dic1_data[i]:
            p_count_data += p_dic1_data[i][j]
        for j in p_dic1_data[i]:
            word_data[j]=p_dic1_data[i][j]/p_count_data
        p_freq_data[i] = State(DiscreteDistribution(word_data),name = i)

    model_data = HiddenMarkovModel('pos_tag_data')

    #共有词性多少个
    count_data = 0
    for i in p_dic1_data:
        for j in p_dic1_data[i]:
            count += p_dic1_data[i][j]

    states_data = [] #pos tag
    for i in p_freq_data:
        states.append(p_freq_data[i])

    model_trn.add_states(states_data)

    for i in p_freq_data:
        model_data.add_transition(model_data.start, p_freq_data[i], p_data.count(i)/count_data)

    #transition matrix
    p_sentence_data =[]
    for i in pair:
        p_string_data = []
        for j in i:
            p_string_data.append(j[1])   
        p_sentence_data.append(p_string_data)

    trans_prob_data = {}
    for s in p_sentence_data:
        numerator_data =0
        for i in range(len(s)-1):
            first_data = s[i]
            second_data = s[i+1]
            denom_data = p_data.count(first)
        if any([first_data,second_data] == s[i:i+2] for i in range(len(s) - 1)):
            numerator_data += 1
        trans_prob_data[(first_data,second_data)] = numerator_data/denom_data


    for i in p_freq_data:
        prob_data=0
        for j in p_freq_data:
            if (i,j) not in trans_prob_data.keys():
                trans_prob_data[(i,j)] = 0
            model_data.add_transition(p_freq_data[i],p_freq_data[j],trans_prob_data[(i,j)])

            prob_data+=trans_prob_data[(i,j)]
        if prob_data<1:
            model_data.add_transition(i,model_data.end,1-prob_data)
    return model_data.bake()